## cURL before you can walk

This tutorial will cover the basics of interacting with the ```monero-wallet-rpc```, introducing ```cURL``` and the use of the ```python-monerorpc``` library which makes the whole process alot easier.

>a remote procedure call (RPC) is when a computer program causes a procedure (subroutine) to execute in a different address space (commonly on another computer on a shared network), which is coded as if it were a normal (local) procedure call,
>
> -- <cite>[Remote Procedure Call](https://en.wikipedia.org/wiki/Remote_procedure_call)</cite>

The RPC wallet allows us to do everything the cli wallet can do, but from a distance - and we can control / automate it with scripts.

It requires a node address, port, and wallet to be started:

```
./monero-wallet-rpc --daemon-address localhost:38081 --rpc-bind-port 38083
```
Now lets send an RPC packet using cURL:

```
curl http://localhost:38083/json_rpc -d \
'{"jsonrpc":"2.0","id":"0","method":"create_address", \
"params":{"account_index":0,"label":"My new address"}}' \
-H 'Content-Type: application/json'
```

Notice the port ```38083``` is the same value as ```--rpc-bind-port```. The method we used is called ```create_address``` and inside the ```params``` we are telling it that the address should be created with an ```account_index``` of ```0``` (the main wallet account) and to set the ```label``` of it to ```My new address```.

Referring to the documentation [here](https://www.getmonero.org/resources/developer-guides/wallet-rpc.html#create_address) the ```Inputs``` are contained inside ```params```.

The output:
```
{
  "id": "0",
  "jsonrpc": "2.0",
  "result": {
    "address": "7BG5jr9QS5sGMdpbBrZEwVLZjSKJGJBsXdZLt8wiXyhhLjy7x2LZxsrAnHTgD8oG46ZtLjUGic2pWc96GFkGNPQQDA3Dt7Q",
    "address_index": 5
  }
}
```
An example of parsing the data inside ```result``` with bash using ```jq``` can be seen here in CryptoGrampys' [android termux node](https://github.com/CryptoGrampy/android-termux-monero-node/blob/713297615d99665e868d52deceede74bd47534d6/src/install-monerod-in-termux.sh#L381) 

## Time to walk (slither?)

To replicate cURL in Python we could just send a json RPC packet using the ```requests``` library but it would soon become a mess. [```python-monerorpc```](https://github.com/monero-ecosystem/python-monerorpc) simplifies the process and is used as follows:

In [ ]:
# Run this example a few times. 
# Notice how we are given a new address each time and address_index number increases. 
#An address created at index 1 will always be the same as Monero wallets are determinalistic. 

from monerorpc.authproxy import AuthServiceProxy, JSONRPCException
import pprint
rpc_connection = AuthServiceProxy(service_url="http://localhost:38083/json_rpc")
params = {
    "account_index": 0,
    "label": "My new address"
}
info = rpc_connection.create_address(params)
#print the result
pprint.pprint(info)
#lets access the address
the_address = info["address"]
the_index = info["address_index"]

print(f"\nHello world, we just created {the_address} at index {the_index}")

Notice how in the above example the ```method``` name comes after ```rpc_connection``` and pass the params (```Inputs```) to it. If no inputs are requires then ```params``` does not have to be set / passed. 

For the final example i am going to use [get_transfers](https://www.getmonero.org/resources/developer-guides/wallet-rpc.html#get_transfers). If you look at the docs you'll see a huge list of ```Inputs``` but most of them are optional. I want to find only incoming transfers from account index 0 so my params will look like:
```
params = {
    "in": True,
    "account_index": 0
}
```

In [13]:
from monerorpc.authproxy import AuthServiceProxy, JSONRPCException
import pprint
rpc_connection = AuthServiceProxy(service_url="http://localhost:38083/json_rpc")
params = {
    "in": True,
    "account_index": 0
}
info = rpc_connection.get_transfers(params)
#print the result
pprint.pprint(info)

#lets loop and access the data
for incoming_transaction in info["in"]:
    the_address = incoming_transaction["address"]
    the_txid = incoming_transaction["txid"]
    the_confs = incoming_transaction["confirmations"]
    print(f"\nAddress: {the_address}\nTXID: {the_txid}\nConfirmations: {the_confs}\n")

{'in': [{'address': '54hzhv2oXnHKHmbDhjuhN8Dz4XjUaxCuBBCWRRL5DPoWW7gDkGvetwB1bAkM4jBxnhLAawfL9sC4dibZGBhCtAMASxCuRvn',
         'amount': 4345000000000,
         'amounts': [2000000000,
                     3000000000,
                     340000000000,
                     1000000000000,
                     3000000000000],
         'confirmations': 1412,
         'double_spend_seen': False,
         'fee': 108590000,
         'height': 1081073,
         'locked': False,
         'note': '',
         'payment_id': '0000000000000000',
         'subaddr_index': {'major': 0, 'minor': 0},
         'subaddr_indices': [{'major': 0, 'minor': 0}],
         'suggested_confirmations_threshold': 1,
         'timestamp': 1651153825,
         'txid': 'de817cdfc551371bd2bc5b4e642162f41d5e53c2803a070e5c5fb95e6bb2735a',
         'type': 'in',
         'unlock_time': 0},
        {'address': '54hzhv2oXnHKHmbDhjuhN8Dz4XjUaxCuBBCWRRL5DPoWW7gDkGvetwB1bAkM4jBxnhLAawfL9sC4dibZGBhCtAMASxCuRvn',
         'amo

## Exercise 1

Using the method [```get_transfer_by_txid```](https://www.getmonero.org/resources/developer-guides/wallet-rpc.html#get_transfer_by_txid) print out the ```info``` returned. This method has 2 inputs , the optional one should be ignored, so it only needs to be given a ```txid``` which must be existing in the wallet already (obtained from the example above)